In [22]:
import pandas as pd
import numpy as np
import requests
import regex as re
import json

In [2]:
filepath = 'C:/Users/Tim/Desktop/lighthouse/w8/d4/'
file = 'nlpdataset2.txt'

with open(filepath+file) as f:
    read_data = f.read()

In [3]:
df = pd.read_json(read_data, lines=True)

In [4]:
df.head()

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,lang,timestamp_ms,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,possibly_sensitive,extended_tweet
0,2021-02-26 01:00:56+00:00,1365104527540641799,1365104527540641792,@luatds Thanks! :),"[8, 18]","<a href=""http://twitter.com/download/android"" ...",False,1.365103e+18,1.365103e+18,1.355223e+18,...,en,2021-02-26 01:00:56.438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-26 01:01:56+00:00,1365104778540425219,1365104778540425216,RT @twtselfieupdate: hey hey hey! it’s the 26t...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,...,en,2021-02-26 01:01:56.281,{'created_at': 'Fri Feb 26 01:01:41 +0000 2021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-26 01:01:56+00:00,1365104780704686082,1365104780704686080,RT @pulte: I’m going to give one random person...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,...,en,2021-02-26 01:01:56.797,{'created_at': 'Thu Feb 25 19:09:00 +0000 2021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-26 01:01:56+00:00,1365104780649979905,1365104780649979904,@buckeyegirll Hi Michelle! How are you? So nic...,"[14, 75]","<a href=""https://about.twitter.com/products/tw...",False,1.365105e+18,1.365105e+18,1.206092e+08,...,en,2021-02-26 01:01:56.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-26 01:01:56+00:00,1365104780775841794,1365104780775841792,@Karin11717512 @Acosta Air strikes are good wh...,"[23, 79]","<a href=""http://twitter.com/download/android"" ...",False,1.365101e+18,1.365101e+18,1.332682e+18,...,en,2021-02-26 01:01:56.814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['id'].nunique()

1325

In [6]:
df['label'] = np.where(df['text'].str.contains(":\)"),'1','0')

In [7]:
df[['label','text']].head()

,label,text
0,1,@luatds Thanks! :)
1,0,RT @twtselfieupdate: hey hey hey! it’s the 26t...
2,0,RT @pulte: I’m going to give one random person...
3,1,@buckeyegirll Hi Michelle! How are you? So nic...
4,1,@Karin11717512 @Acosta Air strikes are good wh...


In [8]:
ds = ['label','text']
dataset = df[ds]

In [9]:
dataset.head()

,label,text
0,1,@luatds Thanks! :)
1,0,RT @twtselfieupdate: hey hey hey! it’s the 26t...
2,0,RT @pulte: I’m going to give one random person...
3,1,@buckeyegirll Hi Michelle! How are you? So nic...
4,1,@Karin11717512 @Acosta Air strikes are good wh...


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset.text,dataset.label,train_size=0.7,random_state=123)

In [11]:
import nltk
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))

import string
import spacy

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn import metrics

In [195]:
def remove_punc(x):
    pncmap = str.maketrans("","", string.punctuation)
    stripped = [i.translate(pncmap) for i in x]
    return stripped

def lower(x):
    lower = [i.lower() for i in x]
    return lower

# def remove_stopwords(x):
#     words = [i for i in x if not i in stop_words]
#     return words

nlp = spacy.load("en_core_web_sm")

def normalize_text(documents, 
                   min_token_len=2, 
                   irrelevant_pos=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Keyword arguments:
    documents -- (np.array[str]) the list of documents to be preprocessed
    min_token_len -- (int) min_token_length required
    irrelevant_pos -- (list) a list of irrelevant pos tags
    
    Returns: np.array[str] the normalized documents
    """
    normalized_documents = []
    
    for text in documents:
        # Remove Emails
        # text = remove_punc(text)
        
        #lower
        # text = lower(text)

        # Remove extra space characters
        # text = re.sub(r'\s+', ' ', text)

        # Remove distracting characters
        # text = re.sub(r'''[\*\~]+''', "", text)

        # Remove Emails
        text = re.sub(r'\S*@\S*\s?', '', text)

        # Remove extra space characters
        text = re.sub(r'\s+', ' ', text)

        # Remove distracting characters
        text = re.sub(r'''[\*\~]+''', "", text)        
        
        doc = nlp(text)
        clean_text = []

        for token in doc:
            if (token.is_stop == False # Check if it's not a stopword
                and token.is_alpha): # Check if it's an alphanumerics char
                #and len(token) > min_token_len # Check if the word meets minimum threshold
                #and token.pos_ not in irrelevant_pos): # Check if the POS is in the acceptable POS tags
                lemma = token.lemma_ # Take the lemma of the word
                clean_text.append(lemma)
                
        clean_text = ' '.join(clean_text)
        normalized_documents.append(clean_text)
#         normalized_documents.append(text)
        
    normalized_documents = np.array(normalized_documents)
    return normalized_documents

# Create a Transformer from the function so that we can use it in a Pipeline
normalizer = FunctionTransformer(normalize_text)

In [196]:
print(f'Prior to normalization:\n{x_train[0]}')
print(f'After normalization:\n{normalizer.transform(x_train[0:1])}')

Prior to normalization:
@luatds Thanks! :)
After normalization:
['think fall asleep finger weird finger numb tingly']


In [188]:
doc = nlp('Thanks! :)')
clean_text=[]
for token in doc:
    if (token.is_stop == False # Check if it's not a stopword
        and token.is_alpha # Check if it's an alphanumerics char
        and len(token) > 2 # Check if the word meets minimum threshold
        and token.pos not in ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']): # Check if the POS is in the acceptable POS tags
        lemma = token.lemma_ # Take the lemma of the word
        clean_text.append(lemma)
        print(lemma)

thank


In [181]:
for token in doc:
    print(token.lemma_)

thank
!
:)


In [193]:
def normalized_text(documents, min_token_len=2, irrelevant_pos=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']):
    normalized_documents = []
    for text in documents:
        # Remove Emails
        text = re.sub(r'\S@\S\s?', '', text)
        # Remove extra space characters
        text = re.sub(r'\s+', ' ', text)
        # Remove distracting characters
        text = re.sub(r'''[*~]+''', "", text)
        print(text)
        doc = nlp(text)
        clean_text = []
        for token in doc:
            print(token)
            if (token.is_stop == False # Check if it's not a stopword
                and token.is_alpha # Check if it's an alphanumerics char
                and len(token) > 2 # Check if the word meets minimum threshold
                and token.pos not in ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']): # Check if the POS is in the acceptable POS tags
                lemma = token.lemma_ # Take the lemma of the word
                clean_text.append(lemma)
                print(clean_text)

        clean_text = ' '.join(clean_text)
        normalized_documents.append(clean_text) 

    normalized_documents = np.array(normalized_documents)
    return normalized_documents

normalized_text(['@luatds Thanks! :)'])

@luatds Thanks! :)
@luatds
Thanks
['thanks']
!
:)


array(['thanks'], dtype='<U6')

In [127]:
vectorizer = CountVectorizer()#min_df=2, max_df=0.95)
tftrans = TfidfTransformer(use_idf=True,smooth_idf=True)

In [128]:
preprocess = Pipeline([
    ('normalizer', normalizer),
    ('vectorizer', vectorizer),
    ('tftrans', tftrans)
])

In [129]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [130]:
from sklearn import set_config

set_config(display='diagram')

In [131]:
model_pipe = Pipeline([('preprocessor', preprocess), 
                     ('model', lr)])
model_pipe.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('normalizer',
                                  FunctionTransformer(func=<function normalize_text at 0x000002DB05B6DD30>)),
                                 ('vectorizer', CountVectorizer()),
                                 ('tftrans', TfidfTransformer())])),
                ('model', LogisticRegression())])

In [132]:
y_pred = model_pipe.predict(x_test)
acc = metrics.r2_score(y_test, y_pred)
print(f'Test set accuracy: {acc}')
cm = metrics.confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix: \n{cm}')

Test set accuracy: -0.29672600127145565
Confusion Matrix: 
[[167  75]
 [ 48 108]]
